In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from PIL import Image

from scribbles_creator import *
from scribbles_helpers.cellpose_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define parameters

In [4]:
# Which scribbles to use
mode = "all"
bins = [0.1, 1] #, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
scribble_width = None #2 #1
# all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
# suff = all_suff[:1]
s = "a"

# Which prediction models to use
convpaint_layers = [[0], [0,2]]
convpaint_scalings = [[1], [1,2]]#, [1,2,4], [1,2,4,8]]
use_dino = True
use_ilastik = True

time_seed = 123

# Where to find and save the data
folder_path = "./cellpose_train_imgs"

Loop and analyse

In [10]:
df_list = []

for img_num in [0]: #range(0, 540, 10):
    for bin in bins:
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        for layers in convpaint_layers:
            for scalings in convpaint_scalings:
                print(f"ConvPaint: layers {layers}, scalings {scalings}")
                pred_tag = generate_convpaint_tag(layers, scalings)
                # res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag=pred_tag, show_res=False)
                res = time_cellpose(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag=pred_tag, layer_list=layers, scalings=scalings, random_state=time_seed)
                df_list.append(res)
        if use_ilastik:
            print("Ilastik")
            # res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="ilastik", show_res=False)
            res = time_cellpose(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="ilastik", random_state=time_seed)
            df_list.append(res)
        if use_dino:
            print("DINO")
            # res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="dino", show_res=False)
            res = time_cellpose(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="dino", random_state=time_seed)
            df_list.append(res)

df = pd.concat(df_list, ignore_index=True)
df[:5]

IMG 0: bin 0.1, suff a
ConvPaint: layers [0], scalings [1, 2]
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
Ilastik
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
DINO
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)


C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


IMG 0: bin 1, suff a
ConvPaint: layers [0], scalings [1, 2]
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
Ilastik
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)
DINO
Active channels: R=True, G=True, B=False --> Removed 1 channel(s) --> shape: (2, 383, 512)


C:\Users\roman\AppData\Local\Temp\ipykernel_7196\4162754920.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_list, ignore_index=True)


,img_num,prediction tag,prediction type,layers,scalings,scribbles mode,scribbles bin,suffix,class_1_pix_gt,class_2_pix_gt,...,perc. labelled,t_load,t_features_full,t_features_train,t_train,t_pred,t_selfpred,image,ground truth,scribbles
0,0,convpaint_l-0_s-1-2,convpaint,0,1-2,all,0.1,a,101577,94519,...,0.093832,1.543712,0.104062,0.519687,0.312722,1.198421,2.030830,./cellpose_train_imgs/000_img.png,./cellpose_train_imgs/000_ground_truth.png,./cellpose_train_imgs/000_scribbles_all_00100_...
1,0,ilastik,classical filters,None,None,all,0.1,a,101577,94519,...,0.093832,NaN,0.691719,0.691719,0.314726,0.628522,1.634968,./cellpose_train_imgs/000_img.png,./cellpose_train_imgs/000_ground_truth.png,./cellpose_train_imgs/000_scribbles_all_00100_...
2,0,dino,DINOv2,None,None,all,0.1,a,101577,94519,...,0.093832,1.445783,3.020038,3.020038,0.573309,0.012687,3.606035,./cellpose_train_imgs/000_img.png,./cellpose_train_imgs/000_ground_truth.png,./cellpose_train_imgs/000_scribbles_all_00100_...
3,0,convpaint_l-0_s-1-2,convpaint,0,1-2,all,1.0,a,101577,94519,...,0.980132,1.815006,0.109361,0.532167,2.701932,1.558104,4.792202,./cellpose_train_imgs/000_img.png,./cellpose_train_imgs/000_ground_truth.png,./cellpose_train_imgs/000_scribbles_all_01000_...
4,0,ilastik,classical filters,None,None,all,1.0,a,101577,94519,...,0.980132,NaN,0.516001,0.516001,1.541614,1.067228,3.124843,./cellpose_train_imgs/000_img.png,./cellpose_train_imgs/000_ground_truth.png,./cellpose_train_imgs/000_scribbles_all_01000_...


In [13]:
# df.groupby(['prediction type'])[['accuracy']].mean()
df.groupby(['prediction tag'])[['t_load', 't_features_full', 't_features_train', 't_train', 't_pred', 't_selfpred']].mean()

,t_load,t_features_full,t_features_train,t_train,t_pred,t_selfpred
prediction tag,,,,,,
convpaint_l-0_s-1-2,1.679359,0.106711,0.525927,1.507327,1.378263,3.411516
dino,0.725203,2.960398,2.960398,1.444459,0.016789,4.421646
ilastik,NaN,0.603860,0.603860,0.928170,0.847875,2.379905


Save the data frame in a csv file:

In [4]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
# file_name = f"cellpose_{time_stamp}.csv"
file_name = f"cellpose_times_{time_stamp}.csv"
df.to_csv(file_name, index=False)